In [2]:
import pandas as pd
import numpy as np
import calendar
from time import strptime
from datetime import datetime
import requests
from matplotlib import pyplot as plt

from toby_config import toby_api_key

In [3]:
# Installing bokeh in order to plot on top of a map

import bokeh.io
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, Range1d
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Plasma256




In [4]:
df= pd.read_csv("crime.csv")
df['semi_year'] = df['REPORTED_DATE'].map(lambda x: x)
df['REPORTED_DATE'] = pd.to_datetime(df['REPORTED_DATE'])
df['year'] = df['REPORTED_DATE'].map(lambda x: str(x.year) +'A' if x.month < 7 else str(x.year) +"B")
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,Year,semi_year,year
0,2.016377e+09,2.020000e+15,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 23:31,NaN,2016-06-15,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016,6/15/2016,2016A
1,2.018600e+10,2.020000e+16,2399,0,theft-other,larceny,10/11/2017 12:30,10/11/2017 16:55,2018-01-29,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2018,1/29/2018,2018A
2,2.016600e+10,2.020000e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 20:00,4/25/2016 8:00,2016-04-26,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016,4/26/2016,2016A
3,2.018723e+08,2.020000e+14,2399,0,theft-other,larceny,1/30/2018 19:20,NaN,2018-01-30,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018,1/30/2018,2018A
4,2.017411e+09,2.020000e+15,2303,0,theft-shoplift,larceny,6/22/2017 20:53,NaN,2017-06-23,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017,6/23/2017,2017A


In [5]:
# data munging for 2018A data
df= df[df['IS_TRAFFIC'] == 0]

lat_2018A = df[df['year'] == '2018A']['GEO_LAT'].dropna()
lat_2018A = lat_2018A[lat_2018A > 38]
lat_2018A = lat_2018A[lat_2018A < 41]

lng_2018A = df[df['year'] == '2018A']['GEO_LON'].dropna()
lng_2018A = lng_2018A[lng_2018A > -105.5]
lng_2018A = lng_2018A[lng_2018A < -104.5]


In [6]:
lat_2017A = df[df['year'] == '2017A']['GEO_LAT'].dropna()
lat_2017A = lat_2017A[lat_2017A > 38]
lat_2017A = lat_2017A[lat_2017A < 41]

lng_2017A = df[df['year'] == '2017A']['GEO_LON'].dropna()
lng_2017A = lng_2017A[lng_2017A > -105.5]
lng_2017A = lng_2017A[lng_2017A < -104.5]


In [7]:
# Joining our data into a list so we can loop through

all_lat_lng_data = [[lat_2018A, lng_2018A],[lat_2017A, lng_2017A]]


In [5]:
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,Year,semi_year,year
0,2.016377e+09,2.020000e+15,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 23:31,NaN,2016-06-15,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016,6/15/2016,2016A
1,2.018600e+10,2.020000e+16,2399,0,theft-other,larceny,10/11/2017 12:30,10/11/2017 16:55,2018-01-29,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2018,1/29/2018,2018A
2,2.016600e+10,2.020000e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 20:00,4/25/2016 8:00,2016-04-26,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016,4/26/2016,2016A
3,2.018723e+08,2.020000e+14,2399,0,theft-other,larceny,1/30/2018 19:20,NaN,2018-01-30,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018,1/30/2018,2018A
4,2.017411e+09,2.020000e+15,2303,0,theft-shoplift,larceny,6/22/2017 20:53,NaN,2017-06-23,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017,6/23/2017,2017A


# Maps Plot

In [9]:

map_options = GMapOptions(lat=39.738711, lng=-104.989411, map_type="roadmap", zoom=11)
plot = GMapPlot(
    x_range= Range1d(), y_range= Range1d(), map_options=map_options
)
plot.title.text = "Hey look! It's a scatter plot on a map!"
plot.api_key = toby_api_key

source = ColumnDataSource(
    data=dict(
        lat= lat_2018A,
        lon= lng_2018A,
        size= np.ones(len(lat_2018A)),
        color= np.ones(len(lat_2018A))
    )
)

source2 = ColumnDataSource(
    data=dict(
        lat= lat_201A,
        lon= lng_2018A,
        size= np.ones(len(lat_2018A)),
        color2= 200*np.ones(len(lat_2018A))
    )
)

color_mapper = LinearColorMapper(palette= Plasma256)
circle = Circle(x="lon", y="lat", size="size", fill_color={'field': 'color', 'transform': color_mapper}, fill_alpha=0.5, line_color=None)
plot.add_glyph(source, circle)
plot.add_glyph(source2, circle)

color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))
plot.add_layout(color_bar, 'right')
show(plot)
#plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
#output_file("gmap_plot.html")

#plot.legend.location = "top_left"
#plot.legend.click_policy="mute"


## Map Function

In [13]:
map_options = GMapOptions(lat=39.738711, lng=-104.989411, map_type="roadmap", zoom=11)
plot = GMapPlot(
        x_range= Range1d(), y_range= Range1d(), map_options= map_options
    )

color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                         label_standoff=12, border_line_color=None, location=(0,0))
plot.add_layout(color_bar, 'right')
# plot is our figure, this allows us to plot multiple datasets on same map
# data_num allows us to keep track of which plot this is so we can change the color
def map_plot(plot, lat_data, lng_data, data_num): 
    plot.title.text = "Hey look! It's a scatter plot on a map!"
    plot.api_key = toby_api_key
    source = ColumnDataSource(
        data=dict(
            lat= lat_data,
            lon= lng_data,
            size= np.ones(len(lat_data)),
            color= data_num*np.ones(len(lat_data))
        )
    )
    #color_mapper = CategoricalColorMapper(factors=['hi', 'lo'], palette=[RdBu3[2], RdBu3[0]])
    #color_mapper = LogColorMapper(palette="Viridis5", low=min_median_house_value, high=max_median_house_value)
    color_mapper = LinearColorMapper(palette= Plasma256)
    circle = Circle(x="lon", y="lat", size="size", fill_color={'field': 'color', 'transform': color_mapper}, fill_alpha=0.5, line_color=None)
    plot.add_glyph(source, circle)
    show(plot)
    
    
#     for data, name, color in zip([AAPL, IBM, MSFT, GOOG], ["AAPL", "IBM", "MSFT", "GOOG"], Spectral4):
#     df = pd.DataFrame(data)
#     df['date'] = pd.to_datetime(df['date'])
#     p.line(df['date'], df['close'], line_width=2, color=color, alpha=0.8,
#            muted_color=color, muted_alpha=0.2, legend=name)

#p.legend.location = "top_left"
#p.legend.click_policy="mute"

#output_file("interactive_legend.html", title="interactive_legend.py example")



    #plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
    #output_notebook()

In [14]:
data_num = 0
for each in all_lat_lng_data:
    data_num += 100
    map_plot(plot, each[0], each[1], data_num)
    

In [ ]:
Index(['aggravated-assault', 'all-other-crimes', 'arson', 'auto-theft',
       'burglary', 'drug-alcohol', 'larceny', 'murder',
       'other-crimes-against-persons', 'public-disorder', 'robbery',
       'sexual-assault', 'theft-from-motor-vehicle', 'traffic-accident',
       'white-collar-crime'],
      dtype='object', name='OFFENSE_CATEGORY_ID')